In [7]:
# %% [markdown]
# # Deep SSS Enricher - Jupyter Notebook Versiyonu
# 
# Bu notebook, iki SSS veri setini birleştirir, temizler, zenginleştirir ve RAG-ready hale getirir.

# %% [markdown]
# ## 1. Kurulum ve Import'lar

# %%
# Gerekli kütüphanelerin kurulumu (ilk çalıştırmada bir kez çalıştırın)
# !pip install pandas numpy tqdm sentence-transformers scikit-learn transformers requests faiss-cpu spacy anthropic openai
# !python -m spacy download en_core_web_sm

# %%
import os
import sys
import json
import logging
from datetime import datetime
from pathlib import Path
import time
import re
import hashlib
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm  # Standart tqdm kullan

from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity

try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
        print("✅ spaCy modeli yüklendi")
    except:
        print("⚠️  spaCy modeli bulunamadı, basit mod aktif")
        nlp = None
except ImportError:
    nlp = None
    print("⚠️  spaCy kurulu değil")

import requests

try:
    import faiss
    FAISS_AVAILABLE = True
    print("✅ FAISS kullanılabilir")
except:
    FAISS_AVAILABLE = False
    print("⚠️  FAISS yüklü değil")

print("\n✅ Tüm kütüphaneler yüklendi!")

# %% [markdown]
# ## 2. Konfigürasyon

# %%
# AYARLAR - Buradan değiştirebilirsiniz
CONFIG = {
    'input1': r'..\data\sss_dataset_augmented.json',
    'input2': r'..\data\sss_dataset_heavily_augmented.json',
    'out_dir': '../output',
    'n_paraphrases': 30,
    'provider': 'local',  # 'local', 'anthropic', veya 'openai'
    'api_key': None,  # API key varsa buraya yazın
    'seed': 42,
    'cluster_threshold': 0.3,
    'batch_size': 32
}

print("📋 Konfigürasyon:")
for key, value in CONFIG.items():
    if key != 'api_key':
        print(f"  {key}: {value}")

# %% [markdown]
# ## 3. Yardımcı Fonksiyonlar

# %%
def setup_logger(log_dir: str) -> logging.Logger:
    """Log yapılandırması"""
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"enricher_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
    
    logger = logging.getLogger("DeepEnricher")
    logger.setLevel(logging.INFO)
    logger.handlers = []  # Önceki handler'ları temizle
    
    fh = logging.FileHandler(log_file, encoding='utf-8')
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)
    
    return logger

def normalize_text(text: str) -> str:
    """Metin normalizasyonu"""
    if pd.isna(text):
        return ""
    text = str(text).strip()
    text = text.encode('utf-8', 'ignore').decode('utf-8')
    text = re.sub(r'\s+', ' ', text)
    return text

# %%
def load_dataset(file_path: str) -> pd.DataFrame:
    """Dosyayı otomatik algılayarak yükle"""
    print(f"📂 Yükleniyor: {file_path}")
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Dosya bulunamadı: {file_path}")
    
    ext = os.path.splitext(file_path)[1].lower()
    
    if ext == '.csv':
        df = pd.read_csv(file_path, encoding='utf-8')
    elif ext == '.jsonl':
        df = pd.read_json(file_path, lines=True, encoding='utf-8')
    elif ext == '.json':
        try:
            df = pd.read_json(file_path, encoding='utf-8')
        except:
            df = pd.read_json(file_path, lines=True, encoding='utf-8')
    else:
        raise ValueError(f"Desteklenmeyen format: {ext}")
    
    print(f"✅ {len(df)} satır yüklendi")
    return df

def merge_and_deduplicate(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """İki veri setini birleştir ve tekrarları sil"""
    print("\n🔗 Veri setleri işleniyor...")
    
    # Birleştir
    combined = pd.concat([df1, df2], ignore_index=True)
    print(f"Toplam satır (birleştirme sonrası): {len(combined)}")
    
    # Sütun adlarını normalize et
    combined.columns = combined.columns.str.lower().str.strip()
    
    # all_questions'ı expand et (her soruyu ayrı satır yap)
    expanded_rows = []
    
    for idx, row in tqdm(combined.iterrows(), total=len(combined), desc="Expanding questions"):
        all_questions = row['all_questions']
        
        # Liste formatını kontrol et
        if isinstance(all_questions, str):
            try:
                all_questions = eval(all_questions)
            except:
                all_questions = [all_questions]
        
        # Dict listesi mi yoksa string listesi mi?
        if isinstance(all_questions, list) and len(all_questions) > 0:
            if isinstance(all_questions[0], dict):
                # Dict formatı: [{'question': '...', 'method': '...'}]
                questions = [q.get('question', q.get('text', '')) for q in all_questions]
            else:
                # String listesi
                questions = all_questions
        else:
            questions = [row.get('original_question', '')]
        
        # Her soru için ayrı satır oluştur
        for question in questions:
            expanded_rows.append({
                'question': normalize_text(str(question)),
                'answer': normalize_text(str(row['answer'])),
                'original_question': normalize_text(str(row['original_question'])),
                'category': row.get('category', 'general'),
                'keywords': row.get('keywords', ''),
                'difficulty_level': row.get('difficulty_level', 'medium'),
                'source': 'original'
            })
    
    df = pd.DataFrame(expanded_rows)
    print(f"Toplam soru (expand sonrası): {len(df)}")
    
    # Boş soruları sil
    df = df[df['question'].str.len() > 0]
    df = df[df['answer'].str.len() > 0]
    
    # Exact deduplication
    initial_count = len(df)
    df = df.drop_duplicates(subset=['question', 'answer'], keep='first')
    print(f"🗑️  Exact tekrar silindi: {initial_count - len(df)} satır")
    
    # Created at ekle
    df['created_at'] = datetime.now().isoformat()
    
    df = df.reset_index(drop=True)
    print(f"✅ Temizlenmiş veri: {len(df)} satır")
    
    return df

# %% [markdown]
# ## 4. Embedding ve Kümeleme

# %%
def create_embeddings(texts: List[str], model_name: str = 'sentence-transformers/all-MiniLM-L6-v2') -> np.ndarray:
    """Embedding oluştur"""
    print(f"\n🧠 Embedding model yükleniyor: {model_name}")
    model = SentenceTransformer(model_name)
    
    print(f"🔢 {len(texts)} metin için embedding üretiliyor...")
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
    
    return embeddings, model

def cluster_questions(df: pd.DataFrame, embeddings: np.ndarray, threshold: float) -> pd.DataFrame:
    """Semantik kümeleme"""
    print(f"\n🔍 Semantik kümeleme yapılıyor (threshold={threshold})...")
    
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=threshold,
        linkage='average',
        metric='cosine'
    )
    
    clusters = clustering.fit_predict(embeddings)
    df['cluster_id'] = clusters
    
    n_clusters = len(set(clusters))
    print(f"✅ {n_clusters} küme oluşturuldu")
    
    return df

def select_canonical(group: pd.DataFrame) -> pd.Series:
    """Küme için canonical soru/cevap seç"""
    group = group.copy()
    group['q_len'] = group['question'].str.len()
    canonical_row = group.sort_values('q_len').iloc[0]
    
    unique_answers = group['answer'].unique()
    
    if len(unique_answers) == 1:
        canonical_answer = unique_answers[0]
        needs_review = False
    else:
        canonical_answer = canonical_row['answer']
        needs_review = True
    
    return pd.Series({
        'canonical_question': canonical_row['question'],
        'canonical_answer': canonical_answer,
        'cluster_size': len(group),
        'needs_review': needs_review,
        'initial_sources': '; '.join(group['source'].unique())
    })

def canonicalize_clusters(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Canonical sorular oluştur"""
    print("\n📝 Canonical sorular oluşturuluyor...")
    
    canonical_df = df.groupby('cluster_id').apply(select_canonical).reset_index()
    review_needed = canonical_df[canonical_df['needs_review'] == True].copy()
    
    df = df.merge(
        canonical_df[['cluster_id', 'canonical_question', 'canonical_answer', 'cluster_size']],
        on='cluster_id',
        how='left'
    )
    
    print(f"✅ {len(canonical_df)} canonical soru oluşturuldu")
    print(f"⚠️  {len(review_needed)} küme inceleme gerekiyor")
    
    return df, review_needed

# %% [markdown]
# ## 5. Paraphrase Üretimi

# %%
class ParaphraseGenerator:
    """LLM tabanlı paraphrase üretici"""
    
    def __init__(self, provider: str, api_key: str = None, seed: int = 42):
        self.provider = provider
        self.api_key = api_key
        self.seed = seed
        
    def generate(self, question: str, n: int) -> List[str]:
        """N adet paraphrase üret"""
        if self.provider == 'local':
            return self._local_paraphrase(question, n)
        elif self.provider == 'anthropic':
            return self._anthropic_paraphrase(question, n)
        elif self.provider == 'openai':
            return self._openai_paraphrase(question, n)
        else:
            print(f"❌ Bilinmeyen provider: {self.provider}")
            return []
    
    def _local_paraphrase(self, question: str, n: int) -> List[str]:
        """Basit kural tabanlı paraphrase"""
        synonyms = {
            'nasıl': ['ne şekilde', 'hangi yolla', 'ne biçimde'],
            'nedir': ['ne demektir', 'nedir', 'ne anlama gelir'],
            'yapabilirim': ['yapabilir miyim', 'yapmak mümkün mü', 'yapma imkanı var mı'],
            'öğrenmek': ['bilgi edinmek', 'hakkında bilgi almak', 'öğrenmek'],
            'ne zaman': ['hangi zamanda', 'ne vakit'],
            'nerede': ['hangi yerde', 'nerede'],
            'kim': ['hangi kişi', 'kimler'],
        }
        
        paraphrases = []
        
        for i in range(min(n, 10)):
            new_q = question
            for word, syns in synonyms.items():
                if word in new_q.lower():
                    new_q = new_q.lower().replace(word, np.random.choice(syns))
            
            # Varyasyonlar
            if i % 4 == 0:
                new_q = new_q.capitalize() + "?"
            elif i % 4 == 1:
                new_q = new_q.capitalize() + " hakkında bilgi verir misiniz?"
            elif i % 4 == 2:
                new_q = new_q.capitalize() + " konusunda yardımcı olabilir misiniz?"
            else:
                new_q = new_q.capitalize()
            
            if new_q not in paraphrases:
                paraphrases.append(new_q)
        
        return paraphrases[:n]
    
    def _anthropic_paraphrase(self, question: str, n: int) -> List[str]:
        """Claude API ile paraphrase"""
        url = "https://api.anthropic.com/v1/messages"
        headers = {
            "x-api-key": self.api_key,
            "anthropic-version": "2023-06-01",
            "content-type": "application/json"
        }
        
        paraphrases = []
        batch_size = 5
        
        for batch_idx in range(0, n, batch_size):
            batch_n = min(batch_size, n - batch_idx)
            
            prompt = f"""Aşağıdaki soruyu {batch_n} farklı şekilde yeniden ifade et.

KURALLAR:
- Sorunun ANLAMINI ve AMACINI KESINLIKLE değiştirme
- Yeni bilgi EKLEME veya ÇIKARMA
- Her varyasyon farklı bir tarzda olsun
- Her satıra sadece BIR soru yaz
- Ekstra açıklama yapma

SORU: {question}

ÇIKTI:"""

            payload = {
                "model": "claude-3-5-sonnet-20241022",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": prompt}]
            }
            
            try:
                response = requests.post(url, headers=headers, json=payload, timeout=30)
                
                if response.status_code == 200:
                    content = response.json()['content'][0]['text']
                    lines = [l.strip() for l in content.strip().split('\n') if l.strip() and '?' in l]
                    paraphrases.extend(lines[:batch_n])
                elif response.status_code == 429:
                    print("⏳ Rate limit, bekleniyor...")
                    time.sleep(10)
                
                time.sleep(1)
                
            except Exception as e:
                print(f"❌ Request hatası: {e}")
        
        return paraphrases[:n]
    
    def _openai_paraphrase(self, question: str, n: int) -> List[str]:
        """OpenAI API ile paraphrase"""
        # Benzer implementasyon
        return self._local_paraphrase(question, n)

def generate_paraphrases(df: pd.DataFrame, generator: ParaphraseGenerator, 
                        n_per_question: int, embeddings: np.ndarray, 
                        model: SentenceTransformer) -> pd.DataFrame:
    """Tüm canonical sorular için paraphrase üret"""
    print(f"\n🎭 Paraphrase üretimi ({n_per_question} varyasyon/soru)...")
    
    canonical_questions = df[['cluster_id', 'canonical_question', 'canonical_answer']].drop_duplicates('cluster_id')
    
    all_rows = []
    
    for idx, row in tqdm(canonical_questions.iterrows(), total=len(canonical_questions), desc="Paraphrasing", ncols=80):
        question = row['canonical_question']
        answer = row['canonical_answer']
        cluster_id = row['cluster_id']
        
        paraphrases = generator.generate(question, n_per_question)
        
        if not paraphrases:
            continue
        
        para_embeddings = model.encode(paraphrases, show_progress_bar=False)
        orig_embedding = embeddings[df[df['cluster_id'] == cluster_id].index[0]].reshape(1, -1)
        
        for para, para_emb in zip(paraphrases, para_embeddings):
            similarity = cosine_similarity(orig_embedding, para_emb.reshape(1, -1))[0][0]
            
            if similarity > 0.95 or similarity < 0.70:
                continue
            
            all_rows.append({
                'question': para,
                'answer': answer,
                'canonical_question': question,
                'canonical_id': cluster_id,
                'is_canonical': False,
                'augmentation_method': f'paraphrase_{generator.provider}',
                'confidence': float(similarity),
                'source': 'generated'
            })
    
    print(f"✅ {len(all_rows)} yeni paraphrase oluşturuldu")
    
    return pd.DataFrame(all_rows)

# %% [markdown]
# ## 6. Metadata Zenginleştirme

# %%
def enrich_metadata(df: pd.DataFrame) -> pd.DataFrame:
    """Metadata zenginleştir"""
    print("\n🏷️  Metadata zenginleştiriliyor...")
    
    df['id'] = df.apply(lambda x: hashlib.md5(f"{x['question']}_{x['answer']}".encode()).hexdigest()[:16], axis=1)
    
    if 'canonical_id' not in df.columns:
        df['canonical_id'] = df.get('cluster_id', 0)
    
    if 'is_canonical' not in df.columns:
        df['is_canonical'] = False
    
    # Kategori zaten var, eksikse doldur
    if 'category' not in df.columns or df['category'].isna().any():
        def predict_category(question: str) -> str:
            question_lower = question.lower()
            
            if any(kw in question_lower for kw in ['kayıt', 'üye', 'hesap']):
                return 'account'
            elif any(kw in question_lower for kw in ['ders', 'kurs', 'eğitim', 'bootcamp']):
                return 'courses'
            elif any(kw in question_lower for kw in ['ödeme', 'fiyat', 'ücret']):
                return 'payment'
            elif any(kw in question_lower for kw in ['sertifika', 'diploma']):
                return 'certification'
            elif any(kw in question_lower for kw in ['destek', 'yardım', 'sorun']):
                return 'support'
            else:
                return 'general'
        
        df['category'] = df['category'].fillna(df['question'].apply(predict_category))
    
    def predict_intent(question: str) -> str:
        question_lower = question.lower()
        
        if any(kw in question_lower for kw in ['nasıl', 'ne şekilde', 'how']):
            return 'how_to'
        elif any(kw in question_lower for kw in ['nedir', 'ne demek', 'what']):
            return 'definition'
        elif any(kw in question_lower for kw in ['neden', 'niçin', 'why']):
            return 'reasoning'
        elif any(kw in question_lower for kw in ['ne zaman', 'when']):
            return 'timing'
        else:
            return 'information'
    
    df['intent'] = df['question'].apply(predict_intent)
    
    # Keywords zaten var, eksikse doldur
    if 'keywords' not in df.columns or df['keywords'].isna().any():
        def extract_keywords(text: str) -> str:
            words = re.findall(r'\b\w{4,}\b', text.lower())
            return ', '.join(list(set(words))[:5])
        
        df['keywords'] = df['keywords'].fillna(df['question'].apply(extract_keywords))
    
    # Keywords listeyse stringe çevir
    df['keywords'] = df['keywords'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
    
    df['entities'] = ''
    
    # Difficulty zaten var, eksikse hesapla
    if 'difficulty_level' in df.columns:
        df['difficulty'] = df['difficulty_level']
    else:
        def estimate_difficulty(answer: str) -> str:
            length = len(answer)
            if length < 100:
                return 'easy'
            elif length < 300:
                return 'medium'
            else:
                return 'hard'
        
        df['difficulty'] = df['answer'].apply(estimate_difficulty)
    
    if 'augmentation_method' not in df.columns:
        df['augmentation_method'] = 'original'
    
    if 'confidence' not in df.columns:
        df['confidence'] = 1.0
    
    if 'created_at' not in df.columns:
        df['created_at'] = datetime.now().isoformat()
    
    print("✅ Metadata zenginleştirme tamamlandı")
    
    return df

# %% [markdown]
# ## 7. Ana İşlem

# %%
# Logger kur
log_dir = os.path.join(CONFIG['out_dir'], 'logs')
logger = setup_logger(log_dir)

print("="*80)
print("🚀 DEEP SSS ENRICHER BAŞLATILDI")
print("="*80)

start_time = time.time()

# %% 
# 1. Veri Yükleme
df1 = load_dataset(CONFIG['input1'])
df2 = load_dataset(CONFIG['input2'])

# %%
# 2. Birleştirme ve Temizleme
df = merge_and_deduplicate(df1, df2)

# %%
# 3. Embedding Oluşturma
embeddings, model = create_embeddings(df['question'].tolist())

# %%
# 4. Semantik Kümeleme
df = cluster_questions(df, embeddings, CONFIG['cluster_threshold'])

# %%
# 5. Canonical Sorular
df, review_df = canonicalize_clusters(df)

canonical_mask = df.groupby('cluster_id')['question'].transform(lambda x: x == x.iloc[0])
df['is_canonical'] = canonical_mask

# %%
# 6. Paraphrase Üretimi
if CONFIG['n_paraphrases'] > 0:
    api_key = CONFIG['api_key'] or os.getenv('ANTHROPIC_API_KEY') or os.getenv('OPENAI_API_KEY')
    
    if CONFIG['provider'] != 'local' and not api_key:
        print("⚠️  API key bulunamadı, 'local' moda geçiliyor")
        CONFIG['provider'] = 'local'
    
    generator = ParaphraseGenerator(CONFIG['provider'], api_key, CONFIG['seed'])
    para_df = generate_paraphrases(df, generator, CONFIG['n_paraphrases'], embeddings, model)
    
    df = pd.concat([df, para_df], ignore_index=True)
    print(f"\n📊 Toplam satır (paraphrase sonrası): {len(df)}")

# %%
# 7. Metadata Zenginleştirme
df = enrich_metadata(df)

# %%
# 8. Sonuçları Kaydetme
os.makedirs(CONFIG['out_dir'], exist_ok=True)

df.to_json(os.path.join(CONFIG['out_dir'], 'enriched_dataset.jsonl'), 
           orient='records', lines=True, force_ascii=False)
df.to_csv(os.path.join(CONFIG['out_dir'], 'enriched_dataset.csv'), 
          index=False, encoding='utf-8')

canonical_df = df[df['is_canonical'] == True][['cluster_id', 'canonical_question', 
                                                 'canonical_answer', 'category', 'keywords']]
canonical_df.to_csv(os.path.join(CONFIG['out_dir'], 'canonical_questions.csv'), 
                    index=False, encoding='utf-8')

if len(review_df) > 0:
    review_df.to_csv(os.path.join(CONFIG['out_dir'], 'review_needed.csv'), 
                     index=False, encoding='utf-8')

df.head(50).to_csv(os.path.join(CONFIG['out_dir'], 'preview.csv'), 
                   index=False, encoding='utf-8')

# %%
# 9. Manifest Oluşturma
manifest = {
    'creation_date': datetime.now().isoformat(),
    'total_records': int(len(df)),
    'original_records': int(len(df[df['augmentation_method'] == 'original'])),
    'paraphrased_records': int(len(df[df['augmentation_method'].str.contains('paraphrase', na=False)])),
    'n_clusters': int(df['cluster_id'].nunique()),
    'n_canonical': int(df['is_canonical'].sum()),
    'provider': CONFIG['provider'],
    'n_paraphrases_per_question': CONFIG['n_paraphrases'],
    'seed': CONFIG['seed'],
    'categories': {str(k): int(v) for k, v in df['category'].value_counts().to_dict().items()},
    'intents': {str(k): int(v) for k, v in df['intent'].value_counts().to_dict().items()},
    'augmentation_methods': {str(k): int(v) for k, v in df['augmentation_method'].value_counts().to_dict().items()},
    'difficulty_distribution': {str(k): int(v) for k, v in df['difficulty'].value_counts().to_dict().items()},
    'review_needed_count': int(len(review_df)),
    'processing_time_seconds': round(time.time() - start_time, 2)
}

with open(os.path.join(CONFIG['out_dir'], 'manifest.json'), 'w', encoding='utf-8') as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

print("\n💾 Tüm çıktılar kaydedildi!")

# %% [markdown]
# ## 8. Sonuç Özeti

# %%
print("="*80)
print("✅ İŞLEM TAMAMLANDI")
print("="*80)
print(f"📊 Toplam kayıt: {manifest['total_records']:,}")
print(f"📝 Orijinal kayıt: {manifest['original_records']:,}")
print(f"🎭 Paraphrase kayıt: {manifest['paraphrased_records']:,}")
print(f"🔍 Küme sayısı: {manifest['n_clusters']:,}")
print(f"⭐ Canonical soru: {manifest['n_canonical']:,}")
print(f"⚠️  İnceleme gereken: {manifest['review_needed_count']:,}")
print(f"⏱️  İşlem süresi: {manifest['processing_time_seconds']:.2f} saniye")
print(f"📁 Çıktı dizini: {CONFIG['out_dir']}")
print("="*80)

# %%
# Örnek sonuçları görüntüle
print("\n📋 Örnek Canonical Sorular:")
display(canonical_df.head(10))

print("\n📋 Örnek Paraphrase'ler:")
display(df[df['augmentation_method'].str.contains('paraphrase', na=False)].head(10))

print("\n📊 Kategori Dağılımı:")
display(df['category'].value_counts())

print("\n📊 Intent Dağılımı:")
display(df['intent'].value_counts())

✅ spaCy modeli yüklendi
✅ FAISS kullanılabilir

✅ Tüm kütüphaneler yüklendi!
📋 Konfigürasyon:
  input1: ..\data\sss_dataset_augmented.json
  input2: ..\data\sss_dataset_heavily_augmented.json
  out_dir: ../output
  n_paraphrases: 30
  provider: local
  seed: 42
  cluster_threshold: 0.3
  batch_size: 32
🚀 DEEP SSS ENRICHER BAŞLATILDI
📂 Yükleniyor: ..\data\sss_dataset_augmented.json
✅ 14 satır yüklendi
📂 Yükleniyor: ..\data\sss_dataset_heavily_augmented.json
✅ 14 satır yüklendi

🔗 Veri setleri işleniyor...
Toplam satır (birleştirme sonrası): 28



Expanding questions: 100%|████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 946.26it/s]

Toplam soru (expand sonrası): 812
🗑️  Exact tekrar silindi: 56 satır
✅ Temizlenmiş veri: 756 satır

🧠 Embedding model yükleniyor: sentence-transformers/all-MiniLM-L6-v2


🔢 756 metin için embedding üretiliyor...



Batches: 100%|█████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00,  8.61it/s]


...

,question,answer,original_question,category,keywords,difficulty_level,source,created_at,cluster_id,canonical_question,canonical_answer,cluster_size,is_canonical,canonical_id,augmentation_method,confidence,id,intent,entities,difficulty
756,**bootcamp sertifikası alacak mıyım?**?,"Evet, bootcamp sonunda projenizi başarıyla tam...",NaN,courses,"sertifikası, alacak, bootcamp, mıyım",NaN,generated,NaN,NaN,**Bootcamp Sertifikası Alacak mıyım?**,NaN,NaN,False,0.0,paraphrase_local,0.787344,140d4588eb60f4d5,information,,NaN
757,**bootcamp sertifikası alacak mıyım?** hakkınd...,"Evet, bootcamp sonunda projenizi başarıyla tam...",NaN,courses,"verir, mıyım, misiniz, alacak, sertifikası",NaN,generated,NaN,NaN,**Bootcamp Sertifikası Alacak mıyım?**,NaN,NaN,False,0.0,paraphrase_local,0.750345,de5e6100b415ffce,information,,NaN
758,**bootcamp sertifikası alacak mıyım?** konusun...,"Evet, bootcamp sonunda projenizi başarıyla tam...",NaN,courses,"konusunda, olabilir, mıyım, yardımcı, misiniz",NaN,generated,NaN,NaN,**Bootcamp Sertifikası Alacak mıyım?**,NaN,NaN,False,0.0,paraphrase_local,0.740952,43dc39c20ac1ecc9,information,,NaN
759,**bootcamp sertifikası alacak mıyım?**,"Evet, bootcamp sonunda projenizi başarıyla tam...",NaN,courses,"sertifikası, alacak, bootcamp, mıyım",NaN,generated,NaN,NaN,**Bootcamp Sertifikası Alacak mıyım?**,NaN,NaN,False,0.0,paraphrase_local,0.848164,46da9f877cc008bc,information,,NaN
760,Canlı yayınlar arşivleniyor mu??,"Evet, tüm canlı yayınlar kaydedilmekte ve **Yo...",NaN,general,"arşivleniyor, yayınlar, canlı",NaN,generated,NaN,NaN,Canlı yayınlar arşivleniyor mu?,NaN,NaN,False,22.0,paraphrase_local,0.866781,8e5dded6322d5a7f,information,,NaN
761,Canlı yayınlar arşivleniyor mu? hakkında bilgi...,"Evet, tüm canlı yayınlar kaydedilmekte ve **Yo...",NaN,general,"verir, arşivleniyor, yayınlar, canlı, misiniz",NaN,generated,NaN,NaN,Canlı yayınlar arşivleniyor mu?,NaN,NaN,False,22.0,paraphrase_local,0.798673,d6b30092cedc3ab5,information,,NaN
762,Canlı yayınlar arşivleniyor mu? konusunda yard...,"Evet, tüm canlı yayınlar kaydedilmekte ve **Yo...",NaN,support,"konusunda, olabilir, arşivleniyor, yayınlar, c...",NaN,generated,NaN,NaN,Canlı yayınlar arşivleniyor mu?,NaN,NaN,False,22.0,paraphrase_local,0.790438,de9a60a0749731ce,information,,NaN
763,Canlı yayınlar arşivleniyor mu?,"Evet, tüm canlı yayınlar kaydedilmekte ve **Yo...",NaN,general,"arşivleniyor, yayınlar, canlı",NaN,generated,NaN,NaN,Canlı yayınlar arşivleniyor mu?,NaN,NaN,False,22.0,paraphrase_local,0.859113,059091a1c6046881,information,,NaN
764,Bu projeyi takım halinde mi gerçekleştirmeliyi...,Projeleri tek başınıza ya da en fazla 2 kişili...,NaN,general,"verir, projeyi, takım, halinde, misiniz",NaN,generated,NaN,NaN,Bu projeyi takım halinde mi gerçekleştirmeliyiz?,NaN,NaN,False,16.0,paraphrase_local,0.921963,203802ce140d2a32,information,,NaN
765,Bu projeyi takım halinde mi gerçekleştirmeliyi...,Projeleri tek başınıza ya da en fazla 2 kişili...,NaN,support,"konusunda, projeyi, olabilir, takım, halinde",NaN,generated,NaN,NaN,Bu projeyi takım halinde mi gerçekleştirmeliyiz?,NaN,NaN,False,16.0,paraphrase_local,0.901847,4f554fa9ea0c6cb6,information,,NaN



📊 Kategori Dağılımı:


category
Genel            756
general          126
support           69
certification     38
account           14
courses            8
Name: count, dtype: int64


📊 Intent Dağılımı:


intent
information    717
how_to         157
definition     111
reasoning       18
timing           8
Name: count, dtype: int64